In [1]:
library(Seurat)
library(Signac)
library(genomation)
library(GenomicRanges)
library(BSgenome.Hsapiens.UCSC.hg38)
library(Matrix)
library(parallel)
library(foreach)

Loading required package: SeuratObject

Loading required package: sp

‘SeuratObject’ was built under R 4.3.2 but the current version is
4.3.3; it is recomended that you reinstall ‘SeuratObject’ as the ABI
for R may have changed

‘SeuratObject’ was built with package ‘Matrix’ 1.6.3 but the current
version is 1.6.5; it is recomended that you reinstall ‘SeuratObject’ as
the ABI for ‘Matrix’ may have changed


Attaching package: ‘SeuratObject’


The following object is masked from ‘package:base’:

    intersect


Loading required package: grid

Warning message:
“replacing previous import ‘Biostrings::pattern’ by ‘grid::pattern’ when loading ‘genomation’”
Loading required package: stats4

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following object is masked from ‘package:SeuratObject’:

    intersect


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDupl

Specify file path

In [2]:
gene_gtf_path = "/maps/projects/ralab/data/genome/hg38/gencode.v43.chr_patch_hapl_scaff.annotation.gtf"
abc_genes_path = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/software/scE2G_pipeline/241203/scE2G/ENCODE_rE2G/ABC/reference/hg38/CollapsedGeneBounds.hg38.TSS500bp.bed"
path.pairs.E2G = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/software/scE2G_pipeline/240610/sc-E2G/test/results/K562_IGVF_filter/K562/Kendall/Pairs.tsv.gz"
path.matrix.atac_count = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/software/scE2G_pipeline/240610/sc-E2G/test/results/K562_IGVF_filter/K562/Kendall/atac_matrix.rds"
path.matrix.rna_count = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/analysis/E2G_240503/data/K562_IGVF/1.prepare_data/2.Filter_out_low_quality_cells.240614/rna_count_matrix.csv.gz"
dir.output = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/analysis/E2G_240503/data/K562_IGVF/3.Genome_wide_prediction/Signac/Signac.250210/"

Import candidate E-G pairs

In [3]:
pairs.E2G = readGeneric(path.pairs.E2G,
                        header = T,
                        keep.all.metadata = T)

Import ATAC matrix

In [4]:
matrix.atac = readRDS(path.matrix.atac_count)

Import RNA matrix

In [5]:
matrix.rna_count = read.csv(path.matrix.rna_count,
                            row.names = 1,
                            check.names = F)
matrix.rna_count = Matrix(as.matrix(matrix.rna_count), sparse = TRUE)
matrix.rna_count = matrix.rna_count[,colnames(matrix.atac)]
matrix.rna = NormalizeData(matrix.rna_count)
rm(matrix.rna_count)

Map gene names

In [6]:
extract_attributes <- function(gtf_attributes, att_of_interest){
  att <- unlist(strsplit(gtf_attributes, " "))
  if(att_of_interest %in% att){
    return(gsub("\"|;","", att[which(att %in% att_of_interest)+1]))
  } else {
    return(NA)}
}
map_gene_names <- function(rna_matrix, gene_gtf_path, abc_genes_path){
    library(dplyr)
    library(data.table)
    
	gene_ref <- fread(gene_gtf_path, header = FALSE, sep = "\t") %>%
		setNames(c("chr","source","type","start","end","score","strand","phase","attributes")) %>%
		dplyr::filter(type == "gene")
	gene_ref$gene_ref_name <- unlist(lapply(gene_ref$attributes, extract_attributes, "gene_name"))
	gene_ref$Ensembl_ID <- unlist(lapply(gene_ref$attributes, extract_attributes, "gene_id"))
	gene_ref <- dplyr::select(gene_ref, gene_ref_name, Ensembl_ID) %>%
		mutate(Ensembl_ID = sub("\\.\\d+$", "", Ensembl_ID)) %>% # remove decimal digits 
		distinct()
	
	abc_genes <- fread(abc_genes_path, col.names = c("chr", "start", "end", "name", "score", "strand", "Ensembl_ID", "gene_type")) %>%
		dplyr::select(name, Ensembl_ID) %>%
		rename(abc_name = name) %>%
		left_join(gene_ref, by = "Ensembl_ID") %>%
		group_by(Ensembl_ID) %>% # remove cases where multiple genes map to one ensembl ID
		filter(n() == 1) %>%
		ungroup()

	gene_key <- abc_genes$abc_name
	names(gene_key) <- abc_genes$gene_ref_name

	# remove genes not in our gene universe	
	row_sub <- intersect(rownames(rna_matrix), names(gene_key)) # gene ref names
	rna_matrix_filt <- rna_matrix[row_sub,] # still gene ref names
	rownames(rna_matrix_filt) <- gene_key[row_sub] # converted to abc names

	return(rna_matrix_filt)
}

In [7]:
matrix.rna.rename = map_gene_names(matrix.rna,gene_gtf_path, abc_genes_path)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:Biostrings’:

    collapse, intersect, setdiff, setequal, union


The following object is masked from ‘package:XVector’:

    slice


The following objects are masked from ‘package:GenomicRanges’:

    intersect, setdiff, union


The following object is masked from ‘package:GenomeInfoDb’:

    intersect


The following objects are masked from ‘package:IRanges’:

    collapse, desc, intersect, setdiff, slice, union


The following objects are masked from ‘package:S4Vectors’:

    first, intersect, rename, setdiff, setequal, union


The following objects are masked from ‘package:BiocGenerics’:

    combine, intersect, setdiff, union


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Attaching package: ‘data.table’


The following objects are masked from ‘package:dplyr’:

    between, first, 

Prepare background data

In [8]:
bed.peak = pairs.E2G
mcols(bed.peak) = NULL
bed.peak = unique(bed.peak)
bed.peak = bed.peak[seqnames(bed.peak) %in% paste("chr",c(1:22,"X","Y"), sep = "")]
bed.peak

GRanges object with 153222 ranges and 0 metadata columns:
           seqnames              ranges strand
              <Rle>           <IRanges>  <Rle>
       [1]     chr1   10002642-10003186      *
       [2]     chr1 100028297-100029654      *
       [3]     chr1   10003425-10004218      *
       [4]     chr1 100036849-100037340      *
       [5]     chr1 100037744-100039006      *
       ...      ...                 ...    ...
  [153218]     chrX       989766-990603      *
  [153219]     chrX   99372004-99372431      *
  [153220]     chrX     9940388-9940708      *
  [153221]     chrX     9966874-9967060      *
  [153222]     chrX     9995829-9996183      *
  -------
  seqinfo: 23 sequences from an unspecified genome; no seqlengths

In [9]:
meta.features = RegionStats(bed.peak,
                            genome = BSgenome.Hsapiens.UCSC.hg38)
rownames(meta.features) = paste(bed.peak@seqnames,bed.peak@ranges,sep = "-")
meta.features$seqnames = as.character(bed.peak@seqnames)
meta.features = cbind(meta.features,
                      FindTopFeatures(matrix.atac)[rownames(meta.features),])
meta.features

,AA,AC,AG,AT,CA,CC,CG,CT,GA,GC,⋯,GT,TA,TC,TG,TT,GC.percent,sequence.length,seqnames,count,percentile
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<chr>,<dbl>,<dbl>
chr1-10002642-10003186,46,34,39,29,48,45,7,35,27,38,⋯,25,27,18,44,42,48.80734,545,chr1,48,0.18676822
chr1-100028297-100029654,94,73,97,75,101,83,20,103,81,63,⋯,70,63,89,97,176,43.74080,1358,chr1,102,0.55606245
chr1-10003425-10004218,59,41,44,36,52,63,13,82,34,55,⋯,31,36,51,63,90,46.97733,794,chr1,43,0.11406325
chr1-100036849-100037340,38,32,37,27,38,36,4,46,29,17,⋯,25,30,39,30,35,45.32520,492,chr1,157,0.68748613
chr1-100037744-100039006,50,49,70,33,56,148,125,94,64,142,⋯,62,32,85,73,47,65.24149,1263,chr1,1463,0.95387085
chr1-100044692-100045575,32,25,43,78,39,30,10,64,41,32,⋯,67,65,57,87,168,37.21719,884,chr1,82,0.47106160
chr1-100046070-100046583,15,17,27,25,25,34,9,49,21,30,⋯,36,24,36,51,85,45.33074,514,chr1,44,0.12857162
chr1-100048466-100048972,34,29,21,30,31,29,3,62,21,21,⋯,25,28,45,43,64,41.81460,507,chr1,66,0.37006435
chr1-100050049-100050588,44,27,36,33,27,27,5,43,33,15,⋯,31,36,33,39,70,41.11111,540,chr1,46,0.15809087


Run Signac prediction

In [10]:
PredictionSignac = function(pairs.E2G,
                            data.RNA,
                            data.ATAC,
                            meta.features,
                            n_sample = 200,
                            cor_method = qlcMatrix::corSparse,
                            n.core = 16){
  
  
  my.cluster <- parallel::makeCluster(
    n.core,
    type = "PSOCK"
  )
  doParallel::registerDoParallel(cl = my.cluster)
  
  pairs.E2G =  
    foreach (gene.tmp = unique(pairs.E2G$TargetGene),
             .combine = 'c',
             .packages = c("Signac",
                           "Matrix")) %dopar% {
                             
                             pairs.E2G.tmp = pairs.E2G[pairs.E2G$TargetGene == gene.tmp]
                             
                             data.RNA.tmp.t = t(data.RNA[gene.tmp, 
                                                         , drop = FALSE])
                             data.ATAC.tmp.t = t(data.ATAC[pairs.E2G.tmp$PeakName, 
                                                           , drop = FALSE])
                             
                             coef.result <- cor_method(X = data.ATAC.tmp.t, 
                                                       Y = data.RNA.tmp.t)
                             
                             
                             meta.features.bg <- meta.features[!meta.features$seqnames %in% as.character(seqnames(pairs.E2G.tmp)), ]
                             meta.features.peaks <- meta.features[pairs.E2G.tmp$PeakName, ]
                             bg.peaks <- lapply(X = seq_len(length.out = nrow(x = meta.features.peaks)), 
                                                FUN = function(x) {
                                                  MatchRegionStats(meta.feature = meta.features.bg, 
                                                                   query.feature = meta.features.peaks[x, , drop = FALSE], 
                                                                   features.match = c("GC.percent", "count", "sequence.length"), 
                                                                   n = n_sample, 
                                                                   verbose = FALSE)
                                                })
                             data.ATAC.bg.t <- t(data.ATAC[unlist(x = bg.peaks),, 
                                                           drop = FALSE])
                             bg.coef <- cor_method(X = data.ATAC.bg.t, Y = data.RNA.tmp.t)
                             rownames(bg.coef) <- colnames(data.ATAC.bg.t)
                             for (j in seq(length(pairs.E2G.tmp))) {
                               coef.use <- bg.coef[(((j - 1) * n_sample) + 
                                                      1):(j * n_sample), ]
                               z <- (coef.result[j] - mean(x = coef.use))/sd(x = coef.use)
                               pairs.E2G.tmp[j]$zscore <- z
                             }
                             pairs.E2G.tmp$pvalue = 1 - pnorm(pairs.E2G.tmp$zscore)
                             pairs.E2G.tmp
                           }
  
  parallel::stopCluster(cl = my.cluster)
  
  return(pairs.E2G)
}

In [11]:
pairs.E2G.filter = pairs.E2G[pairs.E2G$TargetGene %in% rownames(matrix.rna.rename) &
                             pairs.E2G$PeakName %in% rownames(matrix.atac)]
mcols(pairs.E2G.filter)[,c("zscore","pvalue")] = NA
pairs.E2G.filter

GRanges object with 10693104 ranges and 5 metadata columns:
             seqnames            ranges strand |  TargetGene
                <Rle>         <IRanges>  <Rle> | <character>
         [1]     chr1 10002642-10003186      * |     AADACL4
         [2]     chr1 10002642-10003186      * |       ACOT7
         [3]     chr1 10002642-10003186      * |      AGTRAP
         [4]     chr1 10002642-10003186      * | APITD1-CORT
         [5]     chr1 10002642-10003186      * |    C1orf158
         ...      ...               ...    ... .         ...
  [10693100]     chrX   9995829-9996183      * |       TBL1X
  [10693101]     chrX   9995829-9996183      * |      TCEANC
  [10693102]     chrX   9995829-9996183      * |      TMSB4X
  [10693103]     chrX   9995829-9996183      * |     TRAPPC2
  [10693104]     chrX   9995829-9996183      * |        WWC3
                           PeakName               PairName    zscore    pvalue
                        <character>            <character> <logical>

In [12]:
start_time <- Sys.time()
pairs.E2G.res = PredictionSignac(pairs.E2G.filter,
                                 matrix.rna.rename,
                                 matrix.atac,
                                 meta.features,
                                 8)
end_time <- Sys.time()
execution_time <- end_time - start_time
execution_time

Time difference of 13.85464 hours

In [13]:
execution_time

Time difference of 13.85464 hours

Save results

In [14]:
dir.create(dir.output,recursive = T)
saveRDS(pairs.E2G.res,
        paste(dir.output,"pairs.E2G.res.rds",sep = "/"))
pairs.E2G.res

GRanges object with 10693104 ranges and 5 metadata columns:
             seqnames            ranges strand |  TargetGene
                <Rle>         <IRanges>  <Rle> | <character>
         [1]     chr1 10002642-10003186      * |     AADACL4
         [2]     chr1 10003425-10004218      * |     AADACL4
         [3]     chr1 10015550-10016393      * |     AADACL4
         [4]     chr1 10016548-10016698      * |     AADACL4
         [5]     chr1 10018554-10019235      * |     AADACL4
         ...      ...               ...    ... .         ...
  [10693100]     chrX 87517607-87517777      * |       KLHL4
  [10693101]     chrX 90453817-90454202      * |       KLHL4
  [10693102]     chrX 90458008-90458800      * |       KLHL4
  [10693103]     chrX 90471409-90471834      * |       KLHL4
  [10693104]     chrX 90474412-90474707      * |       KLHL4
                           PeakName               PairName     zscore
                        <character>            <character>  <numeric>
       

In [15]:
df.output = as.data.frame(pairs.E2G.res)
colnames(df.output)[1] = "chr"
df.output[,"CellType"] = "K562"
data.table::fwrite(df.output,
                   file = paste(dir.output,"pairs.E2G.res.tsv.gz",sep = "/"),
                   row.names = F,
                   quote = F,
                   sep = "\t")
df.output

chr,start,end,width,strand,TargetGene,PeakName,PairName,zscore,pvalue,CellType
<fct>,<int>,<int>,<int>,<fct>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>
chr1,10002642,10003186,545,*,AADACL4,chr1-10002642-10003186,chr1-10002642-10003186_AADACL4,1.28466233,0.09945515,K562
chr1,10003425,10004218,794,*,AADACL4,chr1-10003425-10004218,chr1-10003425-10004218_AADACL4,0.96069420,0.16835298,K562
chr1,10015550,10016393,844,*,AADACL4,chr1-10015550-10016393,chr1-10015550-10016393_AADACL4,-0.12957330,0.55154799,K562
chr1,10016548,10016698,151,*,AADACL4,chr1-10016548-10016698,chr1-10016548-10016698_AADACL4,30.83705252,0.00000000,K562
chr1,10018554,10019235,682,*,AADACL4,chr1-10018554-10019235,chr1-10018554-10019235_AADACL4,-1.45925030,0.92775189,K562
chr1,10032477,10033469,993,*,AADACL4,chr1-10032477-10033469,chr1-10032477-10033469_AADACL4,1.11312206,0.13282800,K562
chr1,10044991,10046648,1658,*,AADACL4,chr1-10044991-10046648,chr1-10044991-10046648_AADACL4,-0.50772851,0.69417812,K562
chr1,10055566,10056226,661,*,AADACL4,chr1-10055566-10056226,chr1-10055566-10056226_AADACL4,-0.91433954,0.81973077,K562
chr1,10064536,10065742,1207,*,AADACL4,chr1-10064536-10065742,chr1-10064536-10065742_AADACL4,0.36660169,0.35695807,K562


In [16]:
sessionInfo()

R version 4.3.3 (2024-02-29)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: Red Hat Enterprise Linux 8.10 (Ootpa)

Matrix products: default
BLAS/LAPACK: /maps/projects/ralab/people/lpm537/software/anaconda3/envs/Notebook_E2G_240505/lib/libopenblasp-r0.3.27.so;  LAPACK version 3.12.0

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

time zone: Europe/Copenhagen
tzcode source: system (glibc)

attached base packages:
 [1] parallel  stats4    grid      stats     graphics  grDevices utils    
 [8] datasets  methods   base     

other attached packages:
 [1] data.table_1.15.2                 dplyr_1.1.4                      
 [3] foreach_1.5.2                     Matrix_1.6